In [2]:
#Estudiantes
# Luis Felipe Calderón Pérez | 2021048663
# Dylan Stef Torres Wslkrt   | 2018135751
#
# Esteban Morales Ureña | 2018171928
from pymongo import MongoClient

client = MongoClient('localhost', 27017, username='root', password='example')

db = client['posts']
posts_collection = db['posts']
posts_collection.drop()




In [3]:
from itertools import islice
import json
from datetime import datetime

with open ('data.jsonl', 'r') as file:
    for line in islice(file,None):
        data = json.loads(line)
        data['post']['timestamp'] = datetime.fromisoformat(data['post']['timestamp'])
        for comment in data['comments']:
            comment['comment']['timestamp'] = datetime.fromisoformat(comment['comment']['timestamp'])
        posts_collection.insert_one(data)

In [9]:
# 1.1 Insertar 5 nuevos documentos de usuario con datos únicos
from itertools import islice
import json
from datetime import datetime


with open ('datos_reales.jsonl', 'r') as file:
    for line in islice(file,None):
        data = json.loads(line)
        data['post']['timestamp'] = datetime.fromisoformat(data['post']['timestamp'])
        for comment in data['comments']:
            comment['comment']['timestamp'] = datetime.fromisoformat(comment['comment']['timestamp'])
        posts_collection.insert_one(data)

In [ ]:
# 1.2 Buscar todos los usuarios con edad mayor a 25.

cursore = posts_collection.find({'user_age': {'$gt': 25}},{'_id': 0})
for doc in cursore:
    print(doc)

In [ ]:
#list(posts_collection.find({}, {'user': 1, "user_email": {'$eq': "tmartin@example.org"}, '_id': 0}))
# 1.3. Buscar un usuario por dirección de correo electrónico

cursore = posts_collection.find({'user_email': {'$eq': 'tmartin@example.org'}},{'_id': 0})
for doc in cursore:
    print(doc)

In [ ]:
# 1.4. Actualizar el contenido de la publicación de un usuario específico.
from bson import ObjectId

posts_collection.update_one({'_id': ObjectId('65f1255d606c54a96079555d')}, {'$set': {'post.content': 'Dylan estuvo aqui'}})

In [5]:
# 1.5. Eliminar un documento de usuario y todos sus comentarios asociados.

posts_collection.delete_many({'user': 'Lindsay Thornton'})

DeleteResult({'n': 3, 'ok': 1.0}, acknowledged=True)

In [ ]:
# 2.1 Buscar todos los usuarios cuyo nombre de usuario contenga la palabra S̈mith.̈ (Expresiones regulares)
puntero=posts_collection.find({ 'user': { '$regex': 'Smith' } })
for doc in puntero:
    print(doc)

In [ ]:
# 2.2 Buscar publicaciones creadas después de una fecha específica (por ejemplo, 2000-01-01).
from datetime import datetime

start_date = datetime(2000, 1, 1)

puntero=posts_collection.find({"post.timestamp": {"$gt": start_date}})
for doc in puntero:
    print(doc)

In [ ]:
# 2.3. Buscar usuarios que aún no han publicado ningún comentario.
cursore = posts_collection.find({ 'comments.comment': { '$exists': False } })

for user in cursore:
    print(user)

In [ ]:
# 3.1 Buscar usuarios que tengan más de 20 años pero menos de 30

cursore = posts_collection.find({'user_age': {'$gt': 20,'$lt':30}},{'_id': 0})
for doc in cursore:
    print(doc)

In [ ]:
# 3.2 Recuperar solo el nombre de usuario y la edad del usuario de todos los documentos. (Proyección)

list(posts_collection.find({}, {'user': 1, "user_age": 1, '_id': 0}))

In [ ]:
# 3.3. Buscar todas las publicaciones y excluir el campo ”comentarios

puntero = posts_collection.find({}, { 'comments': 0 })

for doc in puntero:
    print(doc)

In [ ]:
# 3.5. Buscar comentarios publicados por usuarios mayores de 40 años. (Consultas anidadas)

cursore = posts_collection.find({'comments': {'$elemMatch': {'user_age': {'$gt': 40}}}}, {'comments': 1, '_id': 0})

for doc in cursore:
    print(doc)


In [ ]:
# 4.1 Ordenar todos los usuarios por edad en orden descendente.

#from pymongo import MongoClient

cursore = posts_collection.find({}).sort('user_age', 1)
for doc in cursore:
    print(doc)
                                            

In [ ]:
# 4.2 Buscar los 3 usuarios principales con las publicaciones más recientes (en función de la marca de tiempo).
from datetime import datetime

fecha_actual = datetime.now()

cursore = posts_collection.find({'post.timestamp': {'$lte': fecha_actual}},{'_id': 0}).sort('post.timestamp', -1).limit(3)
for doc in cursore:
    print(doc)


In [ ]:
# 4.3. Omitir los primeros 5 resultados y recuperar los próximos 10 usuarios.

cursore = posts_collection.find().skip(5).limit(10)

for user in cursore:
    print(user)

In [ ]:
# 5.1 
cursore = posts_collection.aggregate([
    {"$group": {"_id": "$user"}},
    {"$group": {"_id": None, "count": {"$sum": 1}}}
],session=None)
for cd in cursore:
    print(cd)

In [ ]:
# 5.2. Encontrar al usuario con el contenido de publicación más largo (en función del número de caracteres).

from bson.son import SON

# Pipeline de agregación para encontrar al usuario con el contenido de publicación más largo
pipeline = [
    {
        '$project': {
            'user': 1,
            'max_post_length': {'$strLenCP': '$post.content'}
        }
    },
    {
        '$sort': SON([('max_post_length', -1)])
    },
    {
        '$limit': 1
    }
]

resultado = posts_collection.aggregate(pipeline)

for doc in resultado:
    print(doc)


In [31]:
# 5.4 Calcular la longitud promedio de los comentarios en todos los comentarios.

#"$unwind": "$comments" descomprime el campo comments, haceindo mas facil despues trabajar con ese campo
#{"$strLenCP": "$comments.comment.text"} accede al comentario que le hicieron al post
cursore = list(posts_collection.aggregate([{"$unwind": "$comments"},{"$group": {"_id": None, "avg_length": {"$avg": {"$strLenCP": "$comments.comment.text"}}}}],session=None))

longitud_promedio = cursore[0]["avg_length"] if cursore else 0

print("Longitud promedio de los comentarios:", longitud_promedio)


Longitud promedio de los comentarios: 159.28579220070904


In [9]:
# 6.1  Incrementar la edad de todos los usuarios en 1.

posts_collection.update_many({},{'$inc': {'user_age': 1}})

UpdateResult({'n': 1435, 'nModified': 1435, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [6]:
#6.2 Establecer el campo ”contenido” de una publicación específica en una cadena vacía.

cursore = posts_collection.find_one_and_update({'post.content': {'$eq': 'los reales nunca moriran brrrrrrr.'}}, {'$set': {'post.content': ''}}, return_document=True)

In [41]:
# 6.3. Eliminar todos los comentarios de la publicación de un usuario.
from pprint import pprint

user = "Joann Bowen"

posts_collection.update_many(
    {'user': user},  # Filtro para identificar las publicaciones del usuario
    {'$unset': {'comments': ''}}  # Operador $unset para eliminar el campo "comments"
)

pprint(posts_collection.find_one({'user': user}))


{'_id': ObjectId('65f32b8209a852acd53cc873'),
 'post': {'content': 'Random content for Joann Bowen Wide take drop. Down goal '
                     'still phone chance tonight. See stock beat significant '
                     'right.\n'
                     'Commercial eye meet impact risk. Every laugh true leave '
                     'chair part because.',
          'timestamp': datetime.datetime(1978, 9, 14, 0, 1, 17, 95000),
          'title': 'Random title for Joann Bowen'},
 'user': 'Joann Bowen',
 'user_age': 19,
 'user_email': 'tmartin@example.org'}


2. Explicar el plan de consulta para encontrar usuarios con edad mayor a 30.


3. Discutir las ventajas y desventajas de almacenar comentarios dentro del documento de usuario

Ventajas: 
1- No hay que hacer joins entre documentos
2- Reduce la necesidad de hacer multiples consultas, ya que los datos están en el mismo objeto

Desventajas:
1- Se complican las consultas, porque hay que acceder a traves de campos en el documento
2- Se hace muy grande el dato del documento haciendo un poco abstracto entenderlo
